# Reorganize representations

In [ ]:
# filename, thermostability, sequence
import os
seqToFile = {}

def addToEntries(set):
    with open(f"data/s_s/{set}/labels.csv", "r") as f:
        firstLine = True

        for line in f:
            if firstLine:
                firstLine = False
                continue
            filename, thermostability, sequence = line.split(", ")
            seqToFile[sequence] = f"{set}/{filename}"
addToEntries("train")
addToEntries("val")

# Merge val and train set

In [ ]:
import shutil
with open("data/s_s/sequences.csv", "w") as f:
    f.write("sequence, filename\n")
    for index, (seq, filename) in enumerate(seqToFile.items()):
        sourcePath = os.path.join("data/s_s", filename)
        targetFilename = f"{index}.pt"
        targetPath = f"data/s_s/{targetFilename}"

        shutil.copyfile(sourcePath, targetPath)
        f.write(f"{seq}, {targetFilename}")

# Create train/eval metadata files

In [ ]:
esm_eval_ids = set()
with open('data/uniref201803_ur50_valid_headers.txt') as txt_file:
    for line in txt_file:
        id = line.split('_')[1].replace('\n','')
        esm_eval_ids.add(id)

def readFasta(filepath='data/full_dataset_sequences.fasta'):
    first = True
    max =0
    dataset = []
    with open(filepath) as fasta:
        for line in fasta:
            if line[0] == '>':
                if first:
                    first = False
                else:
                    dataset.append(entry)
                entry = {}
                header_tokens = line.split(' ')
                entry['id'] = header_tokens[0].replace('>','').split('_')[0]
                entry['header'] = line.replace('\n', '')
                entry['temp'] = float(header_tokens[1].split('=')[1].replace('\n',''))
                entry['sequence'] = ''
            else:
                entry['sequence'] = entry['sequence'] + line.replace('\n','')
                max = len(entry['sequence']) if len(entry['sequence'])> max else max
    
    return dataset


evalDs = []
trainUnfilteredDs = []
trainIds = set()
allIds = set()
dataset = readFasta()
for entry in dataset:
    seq = entry['sequence']
    id = entry["id"]
    allIds.add(id)
    if id in esm_eval_ids:
        evalDs.append(entry)
    else: 
        trainUnfilteredDs.append(entry)
        trainIds.add(id)

# Filter train ds
clusters = {}
with open("data/meltome_PIDE20_clusters.tsv", "r") as f:
    firstLine = True
    for line in f:
        if firstLine:
          firstLine = False
          continue   
        clusterId, proteinId = line.replace("\n", "").split("\t")
        proteinId = proteinId.split('_')[0]
        
        if proteinId in allIds:
            if clusterId in clusters:
                clusters[clusterId].add(proteinId)
            else: 
                clusters[clusterId] = set([proteinId])


for clusterId, proteinids in clusters.items():
  numTrain = 0
  numEval = 0
  for proteinId in proteinids:
    if proteinId in esm_eval_ids:
      numEval += 1
    else: 
      numTrain += 1
    
    if numEval > 0 and numTrain>0:
        for proteinId in proteinids:
            if proteinId in trainIds:
                trainIds.remove(proteinId)

trainDs = [item for item in trainUnfilteredDs if item["id"] in trainIds]


def storeMetadata(ds,set: str):
    with open(f"data/s_s/{set}.csv", "w") as f:
        f.write("sequence, melting point\n")
        for entry in ds:
            f.write(f'{entry["sequence"]}, {entry["temp"]}\n')

storeMetadata(trainDs, "train")
storeMetadata(evalDs, "val")